处理SPOT_1D数据

In [1]:
import pandas as pd
import numpy as np
import re
import itertools
import math
import os

In [2]:
def ss3_feature(seq_ss3):
    ##composition
    seq=list(seq_ss3)
    seq=''.join(seq)
    ss3_data=np.zeros(12)
    doub_name=['C','E','H','CC','CE','CH','EC','EE','EH','HC','HE','HH']
    seq_doub=[a+b for a,b in zip(seq[0:len(seq)-1],seq[1:len(seq)])]
    seq_com=pd.Series(seq_doub)
    seq_tran=pd.Series(list(seq))
    tran_count=seq_tran.value_counts(normalize=True)
    com_count=seq_com.value_counts(normalize=True)
    tran_site=[doub_name.index(i) for i in tran_count.index]
    com_site=[doub_name.index(i) for i in com_count.index]
    np.put(ss3_data,tran_site,tran_count.values)
    np.put(ss3_data,com_site,com_count.values)
   
  # distribution    
    for i in ['C','E','H']:
        c_len=seq.count(i)
        site=np.array([i.start() for i in re.finditer(i,seq)])+1
        if c_len!=0:
            a=math.floor(0.25*c_len)-1
            if a==0:
                a=0
            b=math.floor(0.50*c_len)-1
            if b==0:
                b=0
            c=math.floor(0.75*c_len)-1
            if c==0:
                c=0
            dis_site=[0,a,b,c,-1]
            distri=site[dis_site]/len(seq)
        else:
            distri=np.zeros(5)
        ss3_data=np.hstack((ss3_data,distri))
    return ss3_data ##27dim

In [11]:
SS3_com=['SS3_composition_'+i for i in ['C','E','H']]+['SS3_trainsition_'+i for i in ['CC','CE','CH','EC','EE','EH','HC','HE','HH']]

In [66]:
dd=['C', 'S', 'T', 'H', 'G', 'I','E', 'B', 'CC', 'CS', 'CT', 'CH', 'CG', 'CI', 'CE', 'CB', 'SC', 'SS', 'ST', 'SH', 'SG', 'SI', 'SE', 'SB', 'TC', 'TS',
 'TT', 'TH', 'TG', 'TI', 'TE', 'TB', 'HC', 'HS', 'HT', 'HH', 'HG', 'HI', 'HE', 'HB', 'GC', 'GS', 'GT', 'GH', 'GG', 'GI', 'GE', 'GB', 'IC', 'IS',
 'IT', 'IH', 'IG', 'II', 'IE', 'IB', 'EC', 'ES', 'ET', 'EH', 'EG', 'EI', 'EE', 'EB', 'BC', 'BS', 'BT', 'BH', 'BG', 'BI', 'BE', 'BB']

In [67]:
def ss8_feature(seq_ss8):
    ##composition
    seq=list(seq_ss8)
    seq=''.join(seq)
    ss3_data=np.zeros(72)
    
    ss8='CSTHGIEB'
    doub=[]
    for i in itertools.product(ss8,repeat=2):
        doub.append("".join(i))
    doub_name=list(ss8)+doub
    #
    seq_doub=[a+b for a,b in zip(seq[0:len(seq)-1],seq[1:len(seq)])]
    seq_com=pd.Series(seq_doub)
    seq_tran=pd.Series(list(seq))
    tran_count=seq_tran.value_counts(normalize=True)
    com_count=seq_com.value_counts(normalize=True)
    tran_site=[doub_name.index(i) for i in tran_count.index]
    com_site=[doub_name.index(i) for i in com_count.index]
    np.put(ss3_data,tran_site,tran_count.values)
    np.put(ss3_data,com_site,com_count.values)
   
  # distribution    
    for i in list(ss8):
        c_len=seq.count(i)
        site=np.array([i.start() for i in re.finditer(i,seq)])+1
        if c_len!=0:
            a=math.floor(0.25*c_len)-1
            if a==0:
                a=0
            b=math.floor(0.50*c_len)-1
            if b==0:
                b=0
            c=math.floor(0.75*c_len)-1
            if c==0:
                c=0
            dis_site=[0,a,b,c,-1]
            distri=site[dis_site]/len(seq)
        else:
            distri=np.zeros(5)
        ss3_data=np.hstack((ss3_data,distri))
    return ss3_data #72+8*5=112

In [68]:
def split_n(spot,n):##把特征ASA等特征均分，求均值。
    #n :分为几等份
    seq_feature=np.zeros(n*19) 
    d=0
    for j in range(3,22):
        seq=spot.iloc[:,j]
        if len(seq)%n==0:
            step=int(len(seq/n))
        else:
            step=int(len(seq)/n)+1  #每一份的长度
        for i in range(0,len(seq),step):
            seq_feature[d]=seq[i:i+step].mean()
            d +=1
    return seq_feature   #57dim

In [69]:
def feature(spot,n):
    ss3=ss3_feature(spot.iloc[:,1])
    ss8=ss8_feature(spot.iloc[:,2])
    other=split_n(spot,n)
    structure=np.hstack((ss3,ss8,other))
    return structure #196dim

In [70]:
def number():
    path='F:/zhulin/1datasz1/RNA/predict/predictRBPspot/outputs'
    files=os.listdir(path)
    s=[]
    for file in files:
        a=file[8:]
        s.append(a[:-7])
        b=[int(x) for x in s]
    return b

In [71]:
def structure_name():
    numb=number()
    name=[]
    path='F:/zhulin/1datasz1/RNA/predict/predictRBP'
    for num in numb:
        with open (path+'/'+'predict_{}.fasta'.format(num),'r') as f:
            lines=f.readlines() 
            head=lines[0].strip()
            name.append(head[1:])
    return name
    

In [72]:
def structure_feature(n):
    path='F:/zhulin/1datasz1/RNA/predict/predictRBPspot/outputs'
    numb=number()
    s=[]
    for num in numb:
        with open (path+'/'+'predict_{}.spot1d'.format(num),'r') as f:
            spot_col_names=f.readline().split('\t')
            spot_col_names.pop()
            spot=pd.read_csv(f,delim_whitespace=True,names=spot_col_names)
            spot.drop(axis=0,index=0)
            spot.drop(axis=1,index=0)
        s.append(feature(spot.iloc[:,1:],n))
    featurename=structure_name()
    col=['struc'+str(i) for i in list(range(0,196))]
    structure=pd.DataFrame(s,columns=col,index=featurename)
    return structure
            

In [73]:
struc_feature=structure_feature(3)
outpath="F:/zhulin/1datasz1/RNA/predict/data"
struc_feature.to_csv(os.path.join(outpath,"predict_structure_feature.csv"))